In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# ==============================
# KONFIGURASI
# ==============================
# Daftar kategori berita Kompas Otomotif
URLS = {
    "motor": "https://otomotif.kompas.com/motor",
    "mobil": "https://otomotif.kompas.com/mobil",
    "sport": "https://otomotif.kompas.com/sport",
    "niaga": "https://otomotif.kompas.com/niaga",
    "komunitas": "https://otomotif.kompas.com/komunitas",
    "otopedia": "https://otomotif.kompas.com/otopedia"
}

# Header agar tidak terdeteksi bot
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/122.0.0.0 Safari/537.36"
}

# Batas maksimal berita per kategori
MAX_PER_CATEGORY = 100


# ==============================
# FUNGSI PEMBERSIH TEKS
# ==============================
def bersihkan_article(text):
    """Membersihkan teks dari karakter yang tidak diinginkan"""
    return text.replace('\xa0', ' ').replace('\r', '').replace('\n', ' ').strip()


# ==============================
# FUNGSI AMBIL DATA ARTIKEL
# ==============================
def crawl_information(id_berita, url, label_kategori):
    try:
        result = {}
        res = requests.get(url, headers=HEADERS, timeout=30)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')

        # Simpan kategori
        result['label'] = label_kategori

        # Ambil judul berita
        title_tag = soup.find("h1", class_="read__title")
        result['title'] = title_tag.get_text(strip=True) if title_tag else "NO TITLE"

        # Ambil isi berita
        article_div = soup.find("div", class_="read__content")
        if article_div:
            paragraphs = article_div.find_all("p")
            raw_article = '\n'.join([p.get_text(" ", strip=True) for p in paragraphs if p.get_text(" ", strip=True)])
            result['article'] = bersihkan_article(raw_article)
        else:
            result['article'] = "NO CONTENT"

        return tuple([id_berita] + [v for v in result.values()])
    except Exception as e:
        print(f"[!] Error crawling {url}: {e}")
        return None


# ==============================
# FUNGSI AMBIL SEMUA LINK DARI SATU HALAMAN
# ==============================
def get_article_links(url):
    """Mengambil semua link berita dari halaman kategori"""
    try:
        res = requests.get(url, headers=HEADERS, timeout=30)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')

        links = []
        for a in soup.select("div.article__list a.article__link"):
            link = a.get("href")
            if link and link.startswith("https://otomotif.kompas.com/"):
                links.append(link)

        return list(set(links))  # hilangkan duplikat
    except Exception as e:
        print(f"[!] Error mengambil link dari {url}: {e}")
        return []


# ==============================
# FUNGSI UTAMA CRAWLING MULTI PAGE
# ==============================
def crawl_kompas_otomotif():
    all_data = []
    current_id = 1  # ID berita dimulai dari 1

    for kategori, base_url in URLS.items():
        print(f"\n[INFO] Scraping kategori: {kategori}")
        total_collected = 0
        page = 1

        while total_collected < MAX_PER_CATEGORY:
            # Membuat URL untuk halaman tertentu
            url_page = base_url if page == 1 else f"{base_url}/{page}"
            print(f"[INFO] Mengambil data dari: {url_page}")

            links = get_article_links(url_page)
            if not links:
                print(f"[INFO] Tidak ada link ditemukan di halaman {page}. Berhenti.")
                break

            print(f"[INFO] Ditemukan {len(links)} link di halaman {page}")

            # Proses setiap link berita
            for link in links:
                if total_collected >= MAX_PER_CATEGORY:
                    print(f"[INFO] Sudah mencapai batas {MAX_PER_CATEGORY} berita untuk kategori {kategori}.")
                    break

                data = crawl_information(current_id, link, kategori)
                if data:
                    all_data.append(data)
                    print(f"[INFO] ({total_collected + 1}/{MAX_PER_CATEGORY}) Berhasil ambil: {data[3]}")

                    current_id += 1
                    total_collected += 1

                time.sleep(1)  # delay agar tidak diblokir

            page += 1  # Lanjut ke halaman berikutnya
            time.sleep(2)  # jeda antar halaman

        print(f"[INFO] Total berita kategori '{kategori}': {total_collected}")

    return all_data


# ==============================
# EKSEKUSI
# ==============================
data_berita = crawl_kompas_otomotif()

# Simpan hasil ke Excel
columns = ["id", "label", "title", "article"]
df = pd.DataFrame(data_berita, columns=columns)
filename = f"kompas_otomotif_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

print(f"\nScraping selesai! Data tersimpan di {filename}")



[INFO] Scraping kategori: motor
[INFO] Mengambil data dari: https://otomotif.kompas.com/motor
[INFO] Ditemukan 15 link di halaman 1
[INFO] (1/100) Berhasil ambil: BOGOR, KOMPAS.com - Frontwheel Motors, bengkel custom motor bergaya vintage racing di Cilebut, Bogor, Jawa Barat memiliki pilihan motor custom terbaru dengan sebutan VMX 225. VMX 225 merupakan Vintage MotoCross (VMX) dengan mesin yang lebih besar dari model sebelumnya yakni 150 VMX Sled Megapro. Inspirasi tampilannya tetap sama, dari Rickman Metisse tapi dengan kapasitas mesin yang lebih kecil. Chandra Gunawan, Founder Frontwheel Motors mengatakan, VMX 225 mengadopsi mesin dari Yamaha Scorpio dengan spesifikasi kaki-kaki untuk kebutuhan bermain di berbagai medan. Baca juga: 150 VMX Sled Megapro Buatan Frontwheel Motors, Buktikan Motor Kecil Bisa Keren A post shared by Frontwheel Motorcycle. (@frontwheel.motors) "Didesain ringan, lincah, dan tampilan dibuat vintage. Konstruksi frame juga sedikit berbeda, dengan kaki-kaki dari

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3

# Hilangkan peringatan SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_links_from_page(url):
    """Mengambil semua link dari satu halaman"""
    try:
        response = requests.get(url, verify=False, timeout=50)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Ambil semua tag <a> yang memiliki href
        links = [a['href'] for a in soup.find_all('a', href=True)]
        return links
    except Exception as e:
        print(f"Gagal mengambil {url}: {e}")
        return []

def crawl_and_save(base_url, output_file):
    visited = set()
    data = []
    queue = [base_url]

    while queue:
        current_url = queue.pop(0)
        if current_url in visited:
            continue
        visited.add(current_url)

        print(f"Sedang mengambil: {current_url}")
        links = get_links_from_page(current_url)

        # Simpan hasil ke dalam data
        for link in links:
            if link.startswith('/'):  # jika relatif, tambahkan domain
                link = base_url.rstrip('/') + link
            data.append({
                "page": current_url,
                "link keluar": link
            })

            # Jika link masih dalam domain yang sama dan belum dikunjungi, masukkan ke queue
            if link.startswith(base_url) and link not in visited and link not in queue:
                queue.append(link)

    # Buat DataFrame dan tambahkan kolom id
    df = pd.DataFrame(data)
    df.reset_index(inplace=True)
    df.rename(columns={"index": "id"}, inplace=True)

    # Simpan ke Excel
    df.to_excel(output_file, index=False)
    print(f"\nHasil disimpan ke: {output_file}")

# Jalankan
crawl_and_save("https://fkip.trunojoyo.ac.id/", "output_links.xlsx")


Sedang mengambil: https://fkip.trunojoyo.ac.id/
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=3205
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=65
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=68
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2339
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2341
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2343
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2345
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2947
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=816
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=811
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2868
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2861
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2865
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2871
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=2874
Sedang mengambil: https://fkip.trunojoyo.ac

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3490
Sedang mengambil: https://fkip.trunojoyo.ac.id/?author=1
Sedang mengambil: https://fkip.trunojoyo.ac.id/?cat=28
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3490#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3481
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3481#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3468
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3468#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3455
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3455#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3449
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3449#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3434
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=3434#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?paged=2
Sedang mengambil: https://fkip.trunojoyo.ac.id/?paged=3
Sedang mengambil: https://fkip.trunojoyo.ac.id/?paged=4
Sedang menga

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=816#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?page_id=811#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/wp-content/uploads/2024/07/Sertifikat-Akreditasi-PBSI-2023-2028_compressed.pdf


...

Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20240719
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20240715
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20240626
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20240609
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20240430
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20240426
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20240123
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20231005
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20230918
Sedang mengambil: https://fkip.trunojoyo.ac.id/?author=1&paged=3
Sedang mengambil: https://fkip.trunojoyo.ac.id/wp-content/uploads/2024/07/PENGUMUMAN-PENDAFTARAN-YUDISIUM-GELOMBANG-KEDUA.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Sedang mengambil: https://fkip.trunojoyo.ac.id/wp-content/uploads/2024/07/PENGUMUMAN-PENDAFTARAN-YUDISIUM-GENAP-23-24.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1959
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1959#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1949
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1949#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1944
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1944#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1918
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1918#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1905
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1905#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1901
Sedang mengambil: https://fkip.trunojoyo.ac.id/?p=1901#respond
Sedang mengambil: https://fkip.trunojoyo.ac.id/?paged=14
Sedang mengambil: https://fkip.trunojoyo.ac.id/?portofolio=ruang-baca-fakultas
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20230814
Sedang mengambil: https://fkip.trunojoyo.ac.id/?m=20230809
Sedang mengambil: https://fkip.trunojo

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def ptaa():
    base_url = "https://pta.trunojoyo.ac.id/"
    data = {
        "penulis": [],
        "judul": [],
        "pembimbing_1": [],
        "pembimbing_2": [],
        "abstrak_id": [],
        "abstrak_eng": [],
        "prodi": []
    }

    # Ambil halaman awal sidebar
    r = requests.get(base_url)
    soup = BeautifulSoup(r.content, "html.parser")

    # Ambil semua menu utama (Fakultas)
    menus = soup.select("div#sidebar ul > li > a")
    for menu in menus:
        menu_text = menu.text.strip()
        if menu_text != "Teknik":  # hanya menu Teknik
            continue

        submenu_parent = menu.find_next("ul")
        if not submenu_parent:
            continue

        submenus = submenu_parent.select("li > a")
        submenus = submenus[:9]  # batasi hanya 9 submenu pertama

        for sub in submenus:
            prodi = sub.text.strip()
            link_sub = sub["href"]

            # Filter hanya link dari pta.trunojoyo.ac.id
            if not link_sub.startswith("https://pta.trunojoyo.ac.id/"):
                print(f"Skip prodi {prodi} karena link tidak dari PTA: {link_sub}")
                continue

            print(f"Scraping prodi: {prodi} -> {link_sub}")

            next_pages = [link_sub]  # mulai dari halaman pertama

            # Cek pagination
            r_sub = requests.get(link_sub)
            soup_sub = BeautifulSoup(r_sub.content, "html.parser")
            pagination = soup_sub.select("ol.pagination li a.pag_button[href]")
            for p in pagination:
                href = p["href"]
                if href.startswith("https://pta.trunojoyo.ac.id/") and href not in next_pages:
                    next_pages.append(href)

            # Loop semua halaman
            for page_url in next_pages:
                r_page = requests.get(page_url)
                soup_page = BeautifulSoup(r_page.content, "html.parser")

                # Ambil semua tombol "selengkapnya"
                details = soup_page.select("a.gray.button")
                for det in details:
                    link_detail = det["href"]

                    # Filter detail juga hanya dari pta.trunojoyo.ac.id
                    if not link_detail.startswith("https://pta.trunojoyo.ac.id/"):
                        continue

                    # Masuk ke halaman detail
                    r_det = requests.get(link_detail)
                    soup_det = BeautifulSoup(r_det.content, "html.parser")

                    # Judul
                    try:
                        judul = soup_det.select_one("div.panel-heading").get_text(strip=True)
                    except:
                        judul = ""

                    # Penulis
                    try:
                        penulis = soup_det.find(string=lambda t: t and "Penulis" in t).split(":")[1].strip()
                    except:
                        penulis = ""

                    # Pembimbing I
                    try:
                        pemb1 = soup_det.find(string=lambda t: t and "Dosen Pembimbing I" in t).split(":")[1].strip()
                    except:
                        pemb1 = ""

                    # Pembimbing II
                    try:
                        pemb2 = soup_det.find(string=lambda t: t and "Dosen Pembimbing II" in t).split(":")[1].strip()
                    except:
                        pemb2 = ""

                    # Abstrak ID
                    abstrak_id = ""
                    abstrak_id_tag = soup_det.find("b", string=lambda t: t and "Abstraksi" in t)
                    if abstrak_id_tag:
                        p_tag = abstrak_id_tag.find_next("p", align="justify")
                        if p_tag:
                            abstrak_id = p_tag.get_text(strip=True)

                    # Abstrak ENG
                    abstrak_eng = ""
                    abstrak_eng_tag = soup_det.find("b", string=lambda t: t and "Abstraction" in t)
                    if abstrak_eng_tag:
                        p_tag = abstrak_eng_tag.find_next("p", align="justify")
                        if p_tag:
                            abstrak_eng = p_tag.get_text(strip=True)

                    # Simpan ke dataset
                    data["judul"].append(judul)
                    data["penulis"].append(penulis)
                    data["pembimbing_1"].append(pemb1)
                    data["pembimbing_2"].append(pemb2)
                    data["abstrak_id"].append(abstrak_id)
                    data["abstrak_eng"].append(abstrak_eng)
                    data["prodi"].append(prodi)

                    time.sleep(0.3)  # delay singkat

    df = pd.DataFrame(data)
    df.to_csv("pta_teknik_semua_pagination.csv", index=False, encoding="utf-8-sig")
    return df

# Jalankan langsung
df = ptaa()
print("Scraping selesai untuk 9 submenu pertama Fakultas Teknik (semua halaman).")
print(df.head())

Scraping prodi: Teknik Industri -> https://pta.trunojoyo.ac.id/c_search/byprod/9
Scraping prodi: Teknik Informatika -> https://pta.trunojoyo.ac.id/c_search/byprod/10
Scraping prodi: Manajemen Informatika -> https://pta.trunojoyo.ac.id/c_search/byprod/11
Scraping prodi: Teknik Multimedia Dan Jaringan -> https://pta.trunojoyo.ac.id/c_search/byprod/19
Scraping prodi: Mekatronika -> https://pta.trunojoyo.ac.id/c_search/byprod/20
Scraping prodi: Teknik Elektro -> https://pta.trunojoyo.ac.id/c_search/byprod/23
Scraping prodi: Sistem Informasi -> https://pta.trunojoyo.ac.id/c_search/byprod/31
Scraping prodi: Teknik Mesin -> https://pta.trunojoyo.ac.id/c_search/byprod/32
Scraping prodi: Teknik Mekatronika -> https://pta.trunojoyo.ac.id/c_search/byprod/33
Skip prodi www.trunojoyo.ac.id karena link tidak dari PTA: http://trunojoyo.ac.id
Skip prodi http://e-journal.dikti.go.id karena link tidak dari PTA: http://e-journal.dikti.go.id
Scraping selesai untuk 9 submenu pertama Fakultas Teknik (semua 